In [ ]:
# advanced
import numpy as np
import pandas as pd
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/HW4_data.zip # FILE PATH

In [ ]:
df_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)
df_val = pd.read_csv("HW4_data/public/val_label.csv", index_col=0)

# 建train 的資料夾 裡面放 Edema 跟 non Edema
import os
os.chdir("/content/HW4_data")
print(os.getcwd())
os.mkdir("train")
os.mkdir("valid")

os.chdir("/content/HW4_data/train")
print(os.getcwd())
os.mkdir("Edema")
os.mkdir("non Edema")

os.chdir("/content/HW4_data/valid")
print(os.getcwd())
os.mkdir("Edema")
os.mkdir("non Edema")

from shutil import copy
# 根據.csv 分類
for i in range(10000):
  from_path = "/content/HW4_data/public/train_"+str(i)+".jpg"
  if df_train['Edema'][i] == 0:
   copy(from_path, "/content/HW4_data/train/non Edema")
  elif df_train['Edema'][i] == 1:
   copy(from_path, "/content/HW4_data/train/Edema")

for i in range(1000):
  from_path = "/content/HW4_data/public/val_"+str(i)+".jpg"
  if df_val['Edema'][i] == 0:
    copy(from_path, "/content/HW4_data/valid/non Edema")
  elif df_val['Edema'][i] == 1:
    copy(from_path, "/content/HW4_data/valid/Edema")

/content/HW4_data
/content/HW4_data/train
/content/HW4_data/valid


In [ ]:
!unzip /content/drive/MyDrive/HW4_data.zip # FILE PATH

In [ ]:
# get Y values (multiple lables)  # train
y_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)    
print(y_train.head())     # printing first five rows of the file
print(y_train.columns)
y_train = np.array(y_train.drop(['Age', 'Gender'], axis=1))

   Atelectasis  Cardiomegaly  Edema  ...  Support Devices  Age  Gender
0            0             0      0  ...                1    2       1
1            0             0      0  ...                1    3       0
2            0             0      0  ...                1    2       0
3            0             1      1  ...                0    3       0
4            0             0      0  ...                0    2       1

[5 rows x 9 columns]
Index(['Atelectasis', 'Cardiomegaly', 'Edema', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Support Devices', 'Age', 'Gender'],
      dtype='object')


In [ ]:
# get Y values (multiple lables)  # val
y_val = pd.read_csv("HW4_data/public/val_label.csv", index_col=0)    
print(y_val.head())     # printing first five rows of the file
print(y_val.columns)
# y_val= np.array(y_val)
y_val = np.array(y_val.drop(['Age', 'Gender'], axis=1))

   Atelectasis  Cardiomegaly  Edema  ...  Support Devices  Age  Gender
0            0             0      0  ...                0    4       0
1            0             0      0  ...                0    3       0
2            0             0      0  ...                0    3       0
3            1             0      1  ...                1    2       1
4            0             0      1  ...                1    3       0

[5 rows x 9 columns]
Index(['Atelectasis', 'Cardiomegaly', 'Edema', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Support Devices', 'Age', 'Gender'],
      dtype='object')


In [ ]:
from shutil import copy
for i in range(10000):
  from_path = "/content/HW4_data/public/train_"+str(i)+".jpg"
  copy(from_path, "/content/HW4_data/train")

for i in range(1000):
  from_path = "/content/HW4_data/public/val_"+str(i)+".jpg"
  copy(from_path, "/content/HW4_data/valid")

In [ ]:
from keras.preprocessing import image
from tqdm import tqdm
SIZE = 128
X_dataset_train = []  
for i in tqdm(range(y_train.shape[0])):
    # img = "/content/HW4_data/public/train_"+str(i)+".jpg"
    img = image.load_img("/content/HW4_data/train/train_"+str(i)+".jpg", target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset_train.append(img)
    
X_train = np.array(X_dataset_train)

X_dataset_val = []  
for i in tqdm(range(y_val.shape[0])):
    # img = "/content/HW4_data/public/valid_"+str(i)+".jpg"
    img = image.load_img("/content/HW4_data/valid/val_"+str(i)+".jpg", target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset_val.append(img)
    
X_val = np.array(X_dataset_val)

100%|██████████| 1000/1000 [00:00<00:00, 2563.91it/s]


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.layers import BatchNormalization

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(SIZE,SIZE,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 16)      1216      
                                                                 
 batch_normalization (BatchN  (None, 124, 124, 16)     64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 62, 62, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 62, 62, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 58, 58, 32)        12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 29, 29, 32)       0

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=64)

Epoch 1/10
157/157 [==============================] - 220s 1s/step - loss: 0.4528 - accuracy: 0.6389 - val_loss: 0.5167 - val_accuracy: 0.3970
Epoch 2/10
157/157 [==============================] - 202s 1s/step - loss: 0.3749 - accuracy: 0.6943 - val_loss: 0.3398 - val_accuracy: 0.7120
Epoch 3/10
157/157 [==============================] - 201s 1s/step - loss: 0.3583 - accuracy: 0.7025 - val_loss: 0.6016 - val_accuracy: 0.1710
Epoch 4/10
157/157 [==============================] - 202s 1s/step - loss: 0.3459 - accuracy: 0.7020 - val_loss: 0.8102 - val_accuracy: 0.7170
Epoch 5/10
157/157 [==============================] - 205s 1s/step - loss: 0.3390 - accuracy: 0.7091 - val_loss: 0.3639 - val_accuracy: 0.7400
Epoch 6/10
157/157 [==============================] - 206s 1s/step - loss: 0.3304 - accuracy: 0.7098 - val_loss: 0.3257 - val_accuracy: 0.6900
Epoch 7/10
157/157 [==============================] - 206s 1s/step - loss: 0.3262 - accuracy: 0.7122 - val_loss: 0.4222 - val_accuracy: 0.5320

In [14]:
history.history['accuracy']

[0.6388999819755554,
 0.6942999958992004,
 0.7024999856948853,
 0.7020000219345093,
 0.7091000080108643,
 0.7098000049591064,
 0.7121999859809875,
 0.711899995803833,
 0.7181000113487244,
 0.72079998254776]

In [15]:
X_dataset_test = []  

for i in tqdm(range(1500)):
    img = image.load_img("/content/HW4_data/private/test_"+str(i)+".jpg", target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset_test.append(img)

100%|██████████| 1500/1500 [00:00<00:00, 2341.24it/s]


In [16]:
import csv
output_dataroot ='STUDENTID_advanced_prediction.csv' 
output_datalist = []
output_datalist =  [' ','Atelectasis','Cardiomegaly','Edema','Lung Opacity','No Finding','Pleural Effusion','Support Devices'] 

os.chdir("/content/drive/MyDrive")
print(os.getcwd())

for fn in range(1500):
  img = image.load_img("/content/HW4_data/private/test_"+str(fn)+".jpg", target_size=(SIZE,SIZE,3))
  img = image.img_to_array(img)
  img = img/255.
  img = np.expand_dims(img, axis=0)
  proba = model.predict(img)  #Get probabilities for each class

  # print(proba[0])
  temp= []
  temp.append(fn)

  max_value = max(proba[0])

  if proba[0][4] != max_value: 
    for j in range(7):
      if proba[0][j]<0.3:
        temp.append(0) 
      elif proba[0][j]>=0.3 and j != 4:
        temp.append(1)
      elif proba[0][j]>=0.3 and j == 4:
        temp.append(0)
  elif proba[0][4] == max_value:
    for j in range(7):
      if j != 4:
        temp.append(0) 
      elif j == 4:
        temp.append(1)
  print(fn)
  print(proba[0])
  temp = np.array(temp)

  output_datalist = np.vstack((output_datalist,temp))
  print(output_datalist)


串流輸出內容已截斷至最後 5000 行。
 ['0' '0' '0' ... '1' '0' '0']
 ['1' '0' '0' ... '1' '0' '0']
 ...
 ['1043' '0' '0' ... '1' '0' '0']
 ['1044' '0' '0' ... '1' '0' '0']
 ['1045' '0' '0' ... '1' '0' '0']]
1046
[5.3438328e-08 6.2311269e-06 6.0439110e-04 1.6068243e-05 9.9949145e-01
 3.2269031e-06 4.7818452e-02]
[[' ' 'Atelectasis' 'Cardiomegaly' ... 'No Finding' 'Pleural Effusion'
  'Support Devices']
 ['0' '0' '0' ... '1' '0' '0']
 ['1' '0' '0' ... '1' '0' '0']
 ...
 ['1044' '0' '0' ... '1' '0' '0']
 ['1045' '0' '0' ... '1' '0' '0']
 ['1046' '0' '0' ... '1' '0' '0']]
1047
[2.2515655e-04 1.3606846e-03 2.7443796e-02 2.8347671e-03 9.7538900e-01
 1.8235147e-03 1.5637171e-01]
[[' ' 'Atelectasis' 'Cardiomegaly' ... 'No Finding' 'Pleural Effusion'
  'Support Devices']
 ['0' '0' '0' ... '1' '0' '0']
 ['1' '0' '0' ... '1' '0' '0']
 ...
 ['1045' '0' '0' ... '1' '0' '0']
 ['1046' '0' '0' ... '1' '0' '0']
 ['1047' '0' '0' ... '1' '0' '0']]
1048
[1.5353865e-10 5.8853288e-08 1.8577533e-05 3.2819403e-07 9.9998736e-

In [17]:
# Write prediction to output csv
with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for row in output_datalist:
        writer.writerow(row)